In [ ]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra import policies

In [ ]:
# Use ExecutionProfiles from version 4.0 onwards

cluster = Cluster(['127.0.1.1',], 
                  port=9043, 
                  load_balancing_policy=policies.RoundRobinPolicy()) 

# Getting a session actually connects to Cassandra
session = cluster.connect('lcmap_local')


In [ ]:
# use prepared statements to cut down on network traffic and parsing load in cassandra

insert_dates = session.prepare('INSERT INTO ard_dates2 (chipx, chipy, dates) VALUES (?, ?, ?);')
insert_dates.consistency_level = ConsistencyLevel.QUORUM                               

# these will both use QUORUM
session.execute(insert_dates.bind([0, 0, [0, 2, 4, 8, 10]]))
session.execute(insert_dates.bind([0, 1, [11, 33, 55, 77, 99]]))

# Additional inserts are actually upserts

session.execute("INSERT INTO ard_dates2 (chipx, chipy, another) VALUES (0, 0, 'brick in the wall')")

In [82]:
# access results via the cursor.  Results are namedtuples by default

for r in session.execute('select * from ard_dates2'):
    print(r)
    print(r.chipx)
    print(r.chipy)
    print(r.dates)

Row(chipx=0, chipy=0, another='brick in the wall', dates=[0, 2, 4, 8, 10])
0
0
[0, 2, 4, 8, 10]
Row(chipx=0, chipy=1, another=None, dates=[11, 33, 55, 77, 99])
0
1
[11, 33, 55, 77, 99]


In [ ]:
session.shutdown()
